In [1]:
#from my_snn.rsnn import RSNN, RSNN_2l, RSNN_2l_ALIF
from my_snn.rsnn_delays import RSNN_d_d, RSNN_d_i, RSNN_d_all
from my_snn.tonic_dataloader import DatasetLoader
from my_snn.utils import train, training_plots, ModelLoader
import torch
device = "cuda:0"

import matplotlib.pyplot as  plt

time_window = 250
batch_size = 256
dataset = 'shd'

DL = DatasetLoader(dataset=dataset, caching='disk', num_workers=0, batch_size=batch_size, time_window=time_window)
test_loader, train_loader = DL.get_dataloaders()

ckpt_dir = 'barridos_shd'

data = test_loader, train_loader

dropout = 0.2

import numpy as np
def test_fn(snn, ckpt_dir, test_loader, ma, epoch):
    if (epoch + 1) % 5 == 0:
        snn.test(test_loader, dropout=dropout)
        last_acc = np.array(snn.acc)[-1,1]
        max_acc = np.max(np.array(snn.acc)[:,1])
        if last_acc == max_acc:
            print(f'saving max acc: {max_acc}')
            snn.save_model(snn.modelname+'_'+ ''.join(str(np.array(snn.acc)[-1,1]).split('.')) , ckpt_dir)

D:\RESEARCHERS\Alberto\Anaconda3\envs\deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_150_3_name = 'shd250_48x2_d_150_3_fs_90087890625'
model_150_6_name = 'shd250_48x2_d_150_6_fs_9052734375'

#snn = ModelLoader(model_150_3_name, ckpt_dir, batch_size, device)
#snn.test(test_loader, dropout = dropout)
#torch.cuda.empty_cache() 

In [3]:
for name in [model_150_3_name, model_150_6_name]:
    for k in [1, 5, 10, 15, 20]:
        snn_pooled = ModelLoader(name, 'shd_soa_baselines', batch_size, 'cuda:0')
        snn_pooled.pool_delays('ho', k=k)
        snn_pooled.test(test_loader, dropout = dropout)
        snn_pooled.train_loss = []
        snn_pooled.test_loss = []
        snn_pooled.acc = []
        snn_pooled.modelname = f'{name}_pooled_{k}'
        train(snn_pooled, data, 1e-3, 50, dropout=0.2, lr_scale=(5.0, 2.0), ckpt_dir=ckpt_dir, test_fn=test_fn, scheduler=(10, 0.95))
        torch.cuda.empty_cache()         

delays: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147]
instance of <class 'my_snn.rsnn_delays.RSNN_d_d'> loaded sucessfully
Test Loss: 3.3166302953447615
Avg spk_count per neuron for all 250 timesteps 9.944972038269043
Test Accuracy of the model on the test samples: 14.111
RSNN_d_d(
  (criterion): CrossEntropyLoss()
  (f0_i): Linear(in_features=700, out_features=48, bias=False)
  (f1_f2): Linear(in_features=2400, out_features=48, bias=False)
  (f2_o): Linear(in_features=2400, out_features=20, bias=False)
)
Total params: 196800
Total mult-adds (M): 49.2
training shd250_48x2_d_150_3_fs_90087890625_pooled_1 for 50 epochs...
Epoch [1/50]
Step [10/31], Loss: 28.64133
Step [20/31], Loss: 27.99216
Step [30/31], Loss: 27.74673
Time elasped: 57.13357639312744
Epoch [2/50]
Step [10/31], Loss: 27.37617
Step [20/31], Loss: 27.16577
St